# Static Mode Analysis: Complete Workflow

This notebook covers the complete static-mode analysis workflow:

1. Loading and validating experimental data
2. Parameter exploration and initial value selection
3. NLSQ fitting with diagnostics
4. Multi-start optimization for robustness
5. Result visualization and physical interpretation

**Use static mode when:** your sample is in equilibrium (no shear/flow) and
you see no angular dependence in C2.

---

## 1. Setup and Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from homodyne.config import ConfigManager
from homodyne.data import load_xpcs_data, validate_xpcs_data
from homodyne.optimization.nlsq import fit_nlsq_jax, fit_nlsq_multistart, MultiStartConfig
from homodyne.utils.logging import get_logger, log_phase

logger = get_logger(__name__)

In [ ]:
# Generate synthetic static-mode data
# (Replace with: data = load_xpcs_data("your_config.yaml"))

rng = np.random.default_rng(seed=123)

# True parameters for validation
TRUE = {
    'D0': 850.0,       # Å²/s
    'alpha': -0.35,    # Mild sub-diffusion
    'D_offset': 0.08,  # Å²/s
    'beta_contrast': 0.15,
    'offset': 1.0,
}

q = 0.054   # Å⁻¹
n_t = 50
n_phi = 8
dt = 0.1
t = dt * np.arange(n_t)
phi_angles = np.linspace(0, 315, n_phi)  # 0° to 315° in steps of 45°

c2 = np.zeros((n_phi, n_t, n_t))
for i_phi in range(n_phi):
    for i_t1 in range(n_t):
        for i_t2 in range(i_t1, n_t):
            t1_val, t2_val = t[i_t1], t[i_t2]
            J = (TRUE['D0'] * (t2_val**(TRUE['alpha']+1) - t1_val**(TRUE['alpha']+1)) / (TRUE['alpha']+1)
                 + TRUE['D_offset'] * (t2_val - t1_val))
            g1_sq = np.exp(-2 * q**2 * J)
            val = TRUE['offset'] + TRUE['beta_contrast'] * g1_sq
            noise = 0.003 * rng.standard_normal()
            c2[i_phi, i_t1, i_t2] = val + noise
            c2[i_phi, i_t2, i_t1] = val + noise  # symmetric

data = {
    'c2_exp': c2,
    't1': t,
    't2': t,
    'phi_angles_list': phi_angles,
    'wavevector_q_list': np.array([q]),
    'sigma': 0.003 * np.ones_like(c2),
    'L': 5.0e6,
    'dt': dt,
}

print(f"Data shape: {c2.shape}  # (n_phi={n_phi}, n_t1={n_t}, n_t2={n_t})")
print(f"q = {q} Å⁻¹,  {n_phi} angles,  dt = {dt} s")

## 2. Data Validation

In [ ]:
# Validate the loaded data
report = validate_xpcs_data(data)

print(f"Data valid: {report.is_valid}")
if report.warnings:
    print(f"Warnings:   {report.warnings}")
if report.issues:
    print(f"Issues:     {report.issues}")

# Check angular dependence (should be minimal for static mode)
lag_idx = 3
c2_at_lag = np.array([c2[i_phi, 0, lag_idx] for i_phi in range(n_phi)])
angular_var_ratio = c2_at_lag.std() / c2_at_lag.mean()
print(f"\nAngular variance ratio at lag={lag_idx}: {angular_var_ratio:.4f}")
if angular_var_ratio < 0.02:
    print("→ No significant angular dependence → static mode is appropriate")
else:
    print("→ Angular dependence detected → consider laminar_flow mode")

## 3. Parameter Space Exploration

Before fitting, it's useful to understand the parameter landscape.

In [ ]:
# Explore how D0 affects the decorrelation curve
def model_c2_1d(t_arr, D0, alpha, D_offset, contrast, offset, q):
    """Simple 1D C2 model (time-averaged, no angle dependence)."""
    c2_vals = []
    t0 = t_arr[0]
    for t2 in t_arr:
        J = (D0 * (t2**(alpha+1) - t0**(alpha+1)) / (alpha+1) + D_offset * (t2 - t0))
        g1_sq = np.exp(-2 * q**2 * J)
        c2_vals.append(offset + contrast * g1_sq)
    return np.array(c2_vals)


fig, axes = plt.subplots(1, 3, figsize=(13, 4))

# Vary D0
ax = axes[0]
for D0_test in [200, 500, 850, 2000, 5000]:
    c2_model = model_c2_1d(t, D0_test, TRUE['alpha'], TRUE['D_offset'],
                            TRUE['beta_contrast'], TRUE['offset'], q)
    ax.semilogx(t, c2_model, label=f'D0={D0_test}')
ax.set_xlabel('t (s)')
ax.set_ylabel('C2')
ax.set_title('Effect of D0')
ax.legend(fontsize=8)

# Vary alpha
ax = axes[1]
for alpha_test in [-1.5, -1.0, -0.5, 0.0, 0.5]:
    c2_model = model_c2_1d(t, TRUE['D0'], alpha_test, TRUE['D_offset'],
                            TRUE['beta_contrast'], TRUE['offset'], q)
    ax.semilogx(t, c2_model, label=f'α={alpha_test}')
ax.set_xlabel('t (s)')
ax.set_title('Effect of alpha')
ax.legend(fontsize=8)

# Vary contrast
ax = axes[2]
for beta_test in [0.05, 0.10, 0.15, 0.25, 0.40]:
    c2_model = model_c2_1d(t, TRUE['D0'], TRUE['alpha'], TRUE['D_offset'],
                            beta_test, TRUE['offset'], q)
    ax.semilogx(t, c2_model, label=f'β={beta_test}')
ax.set_xlabel('t (s)')
ax.set_title('Effect of contrast (β)')
ax.legend(fontsize=8)

plt.suptitle('Parameter Sensitivity Analysis', fontsize=12)
plt.tight_layout()
plt.show()

## 4. Single-Start NLSQ Fit

In [ ]:
import tempfile
config_yaml = """
data:
  file_path: "dummy.h5"
  q_value: 0.054
  dt: 0.1

analysis:
  mode: "static"

optimization:
  method: "nlsq"
  nlsq:
    anti_degeneracy:
      per_angle_mode: "auto"

parameter_space:
  D0:
    initial: 500.0
    bounds: [0.1, 1.0e5]
  alpha:
    initial: -0.5
    bounds: [-2.0, 1.0]
  D_offset:
    initial: 0.1
    bounds: [0.0, 100.0]
"""

with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
    f.write(config_yaml)
    config_path = f.name

config = ConfigManager.from_yaml(config_path)

with log_phase("NLSQ Single-Start"):
    result_single = fit_nlsq_jax(data, config)

print(f"Status:      {result_single.convergence_status}")
print(f"chi^2_nu:    {result_single.reduced_chi_squared:.4f}")
print(f"Time:        {result_single.execution_time:.2f} s")
print()
print(f"{'Param':<12} {'True':>10} {'Fitted':>10} {'Err':>10}")
print('-' * 46)
for i, name in enumerate(['D0', 'alpha', 'D_offset']):
    tv = TRUE[name]
    fv = result_single.parameters[i]
    fe = result_single.uncertainties[i]
    print(f"{name:<12} {tv:>10.4g} {fv:>10.4g} {fe:>10.4g}")

## 5. Multi-Start Optimization

For robust results, run multiple starts and take the best:

In [ ]:
# Multi-start with Latin Hypercube Sampling
ms_config = MultiStartConfig(
    n_starts=10,       # Number of random starting points
    use_lhs=True,      # Latin Hypercube Sampling for coverage
)

with log_phase("NLSQ Multi-Start"):
    result_multi = fit_nlsq_multistart(data, config, ms_config=ms_config)

print(f"Best chi^2_nu:        {result_multi.best_result.reduced_chi_squared:.4f}")
print(f"Starts converged:     {result_multi.n_converged}/{result_multi.n_starts}")
print(f"Best start index:     {result_multi.best_start_idx}")
print()

# Show spread of D0 estimates across starts to assess uniqueness
D0_estimates = [s.result.parameters[0] for s in result_multi.all_starts
                if s.result.convergence_status == 'converged']
print(f"D0 across converged starts: mean={np.mean(D0_estimates):.1f},"
      f" std={np.std(D0_estimates):.1f} Å²/s")
print("(Small std → unique global minimum; large std → multi-modal)")

## 6. Comparing Starting Points

In [ ]:
# Test sensitivity to initial parameters
initial_conditions = [
    {'D0': 100,  'alpha': -1.0, 'D_offset': 0.01,  'label': 'Far below'},
    {'D0': 500,  'alpha': -0.5, 'D_offset': 0.1,   'label': 'Near true'},
    {'D0': 5000, 'alpha':  0.5, 'D_offset': 1.0,   'label': 'Far above'},
    {'D0': 850,  'alpha': -0.3, 'D_offset': 0.05,  'label': 'Very close'},
]

results_ic = []
for ic in initial_conditions:
    init = {k: ic[k] for k in ['D0', 'alpha', 'D_offset']}
    r = fit_nlsq_jax(data, config, initial_params=init)
    results_ic.append({'label': ic['label'], 'result': r})

print(f"{'Starting point':<18} {'D0 fitted':>12} {'alpha':>10} {'chi2_nu':>10} {'status':>12}")
print('-' * 66)
for entry in results_ic:
    r = entry['result']
    print(f"{entry['label']:<18} {r.parameters[0]:>12.1f} {r.parameters[1]:>10.3f}"
          f" {r.reduced_chi_squared:>10.4f} {r.convergence_status:>12}")

## 7. Result Visualization

In [ ]:
best_result = result_multi.best_result
D0_fit = best_result.parameters[0]
alpha_fit = best_result.parameters[1]
D_offset_fit = best_result.parameters[2]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot 1: C2 lag-time curve with model overlay
ax = axes[0]
# Experimental: average over all angles
lag_indices = range(1, n_t - 1)
lag_times = [t[k] for k in lag_indices]
c2_exp_avg = []
for lag_idx in lag_indices:
    n = n_t - lag_idx
    vals = np.array([c2[i_phi, k, k + lag_idx]
                     for i_phi in range(n_phi) for k in range(n)])
    c2_exp_avg.append(np.mean(vals))

# Model (from fitted params)
t0 = t[0]
c2_model_avg = []
for lag_idx in lag_indices:
    t2 = t[lag_idx]
    J = (D0_fit * (t2**(alpha_fit+1) - t0**(alpha_fit+1)) / (alpha_fit+1)
         + D_offset_fit * (t2 - t0))
    g1_sq = np.exp(-2 * q**2 * J)
    c2_model_avg.append(1.0 + 0.15 * g1_sq)  # using estimated contrast

ax.semilogx(lag_times, c2_exp_avg, 'ko', markersize=3, label='Experiment')
ax.semilogx(lag_times, c2_model_avg, 'r-', linewidth=2, label='Model fit')
ax.set_xlabel('Lag time (s)')
ax.set_ylabel('⟨C2⟩')
ax.set_title(f'Fit quality: chi²_nu = {best_result.reduced_chi_squared:.3f}')
ax.legend()

# Plot 2: Parameter comparison
ax = axes[1]
param_names_plot = ['D0', 'alpha', 'D_offset']
true_vals = [TRUE['D0'], TRUE['alpha'], TRUE['D_offset']]
fitted_vals = best_result.parameters[:3]
fitted_errs = best_result.uncertainties[:3]

x = np.arange(3)
width = 0.35
bars = ax.bar(x, [abs(tv) for tv in true_vals], width, label='True', alpha=0.6, color='steelblue')
ax.bar(x + width, [abs(fv) for fv in fitted_vals], width,
       yerr=[abs(e) for e in fitted_errs], capsize=5,
       label='Fitted', alpha=0.8, color='orange')
ax.set_xticks(x + width/2)
ax.set_xticklabels(param_names_plot)
ax.set_yscale('log')
ax.set_ylabel('|Parameter value|')
ax.set_title('True vs Fitted Parameters (absolute value)')
ax.legend()

plt.tight_layout()
plt.show()

print("\nFitted parameters:")
for name, tv, fv, fe in zip(param_names_plot, true_vals, fitted_vals, fitted_errs):
    rel = abs(fv - tv) / abs(tv) * 100 if tv != 0 else float('nan')
    print(f"  {name}: true={tv:.4g}, fitted={fv:.4g} ± {fe:.4g}  ({rel:.1f}% error)")

## 8. Summary

Key takeaways from static mode analysis:

- **D₀** characterizes particle dynamics; compare to Stokes-Einstein for size
- **alpha** reveals diffusion regime (< 0: sub, 0: normal, > 0: super)
- **Anti-degeneracy** (`per_angle_mode: "auto"`) protects physical parameters
- **Multi-start** provides confidence that the global minimum was found
- **chi²_nu ~ 1** indicates a good fit with well-estimated uncertainties

For rigorous uncertainty quantification, proceed to `04_bayesian_inference.ipynb`.

In [ ]:
import os
os.unlink(config_path)